<a href="https://colab.research.google.com/github/lMiaul/SI_TF/blob/feat%2Fjulian/Knmeans_Julian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [13]:
# Cargar el archivo CSV desde Google Drive usando un delimitador diferente
file_path = '/content/drive/MyDrive/SI/B. Generación Anual de residuos municipal_Distrital_2014_2021_0.csv'
data = pd.read_csv(file_path, delimiter=';', encoding='latin1')

In [14]:
# Mostrar las primeras filas del dataset para entender su estructura
print(data.head())

  FECHA_CORTE  N_SEC  UBIGEO REG_NAT DEPARTAMENTO    PROVINCIA     DISTRITO  \
0   9/01/2023      1   10101   SELVA     AMAZONAS  CHACHAPOYAS  CHACHAPOYAS   
1   9/01/2023      2   10102   SELVA     AMAZONAS  CHACHAPOYAS     ASUNCION   
2   9/01/2023      3   10103  SIERRA     AMAZONAS  CHACHAPOYAS       BALSAS   
3   9/01/2023      4   10104  SIERRA     AMAZONAS  CHACHAPOYAS        CHETO   
4   9/01/2023      5   10105  SIERRA     AMAZONAS  CHACHAPOYAS    CHILIQUIN   

   POB_TOTAL  POB_URBANA  POB_RURAL QRESIDUOS_MUN  PERIODO  
0      28423       27548        875       6939,28     2014  
1        291         151        140         47,95     2014  
2       1615         299       1316         69,95     2014  
3        597         388        209         90,84     2014  
4        737         197        540         46,26     2014  


In [15]:
# Verificar los nombres de las columnas en el DataFrame
print("Columnas en el DataFrame:", data.columns)


Columnas en el DataFrame: Index(['FECHA_CORTE', 'N_SEC', 'UBIGEO', 'REG_NAT', 'DEPARTAMENTO',
       'PROVINCIA', 'DISTRITO', 'POB_TOTAL', 'POB_URBANA', 'POB_RURAL',
       'QRESIDUOS_MUN', 'PERIODO'],
      dtype='object')


In [16]:
# Ajustar los nombres de las columnas de interés basándonos en la salida anterior
columns_of_interest = ['QRESIDUOS_MUN', 'POB_TOTAL', 'POB_URBANA', 'POB_RURAL', 'PERIODO']

In [18]:
# Seleccionar las columnas relevantes para el clustering
data_selected = data[columns_of_interest]